# Analyze RCI correlations between cell lines

In [1]:
from datetime import datetime
print(datetime.now())
from platform import python_version
print('Python',python_version())
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt 
import sklearn   # pip install --upgrade scikit-learn
print('sklearn',sklearn.__version__)

2023-01-08 12:27:22.445524
Python 3.10.0
sklearn 1.1.2


In [2]:
try:
    from google.colab import drive
    IN_COLAB = True
    print('Running on CoLab')
    PATH='/content/drive/'
    drive.mount(PATH)
    DATA_DIR=PATH+'My Drive/data/Localization/TrainTest/'  # must end in "/"
except:
    IN_COLAB = False
    DATA_DIR = 'D:/Adjeroh/Localization/TrainTest/'   # Windows
    DATA_DIR = '/Users/jasonmiller/WVU/Localization/TrainTest/'    # Mac
print(DATA_DIR)

/Users/jasonmiller/WVU/Localization/TrainTest/


In [3]:
RCI_FILE =      'CNRCI_noncoding_train_RCI.gc42.csv'
SEQUENCE_FILE = 'CNRCI_noncoding_train_transcripts.gc42.csv'

In [4]:
def load_RCI_data(filepath):
    cell_line_names = []
    cell_line_maps  = []
    with open (filepath,'r') as handle:
        header = None
        for row in handle:
            line = row.strip()
            fields = line.split(',')
            gene_id = fields[0]
            if header is None:
                header = row 
                for i in range(len(fields)): 
                    cell_line_name = fields[i]
                    cell_line_names.append(cell_line_name)
                    cell_line_genes = {}
                    cell_line_maps.append(cell_line_genes)
            else:
                for i in range(len(fields)): 
                    if i>0:  # column 0 is gene_id
                        rci_val = fields[i]
                        if rci_val != "nan":
                            real_value = float(rci_val)
                            cell_line_genes = cell_line_maps[i]
                            cell_line_genes[gene_id] = real_value 
    return cell_line_names, cell_line_maps

In [5]:
filepath = DATA_DIR + RCI_FILE
cl_names,cl_maps = load_RCI_data(filepath)

In [11]:
for i in range(len(cl_names)):
    print(cl_names[i], len(cl_maps[i].keys()))

gene_id 0
A549 1447
H1.hESC 3327
HeLa.S3 919
HepG2 1362
HT1080 936
HUVEC 1488
MCF.7 2156
NCI.H460 610
NHEK 1080
SK.MEL.5 534
SK.N.DZ 603
SK.N.SH 1647
GM12878 1698
K562 958
IMR.90 390
